<a href="https://www.kaggle.com/code/mathewkouch/titanic-passenger-survival-with-decision-tree?scriptVersionId=97437510" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# # Titanic Competitiond - Who survived?

## Steps

### Data Processing
# 1. Load Training, Testing Data [DONE]
# 1.5 Data Transform e.g string to numbers [DONE]
# 2. Create Training, Validation, and Test DataFrames and DataLoaders [DONE]


# Importing Dependencies

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch.nn.functional as F
import math
from torch.optim import lr_scheduler
import matplotlib.pyplot as plt
from sklearn import metrics
import torch
import itertools
from torchvision import models
import torch.optim as optim
from matplotlib.ticker import MaxNLocator
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch import nn
from torch.utils.data import Dataset, DataLoader
import os
from torch.nn import MaxPool2d
from PIL import Image
from sklearn.model_selection import train_test_split
from datetime import date

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


# Utility Functions

In [3]:
def isBoy(person):
    ''' Determines if person is a male and is a boy. Boy is under 16 years old or has "master" in title.
    Arg:
    -person (dictionary) contains the name of the boy (str), and age (float), gender (str)
    '''
    name = person['name']
    gender = person['gender']
    
    name = name.split() # replaces all white space in name with commas
    boy = False
    if gender == 'male':
        for word in name:
            if word.lower() in ['master', 'master.']:
                boy = True
                return boy

    return boy

In [4]:
isBoy({'name':'Patrick Master. F', 'gender':'male'})

True

In [5]:
def get_family_data(data): # Record person by family names
    family_names = {} # dictionary of family names and size of family
    ''' 
    family_names = {
        'count': # number of family members,
        'survived': # number of family members survived
        'boy_survived': # if a boy, count if he survived
    }
    '''
    for idx in range(len(data)):
        full_name = data.Name[idx]
        gender = data.Sex[idx]
        survived = 1 if data.Survived[idx] == 1 else 0
        boy = isBoy({'name':full_name, 'gender': gender})
        full_name = full_name.split()
        family_name = full_name[0][:-1]
        
        if family_name in family_names: # if family name already in dict, then increment family count
            family_names[family_name]['members'].update( {data.Name[idx]: {'id':idx, 'train':True} })
            family_names[family_name]['count'] += 1
            family_names[family_name]['survived'] += survived # increment how many family members survived
            family_names[family_name]['num_female'] += 1 if gender == 'female' else 0
            family_names[family_name]['num_boys'] += 1 if boy else 0
            if boy:
                family_names[family_name]['boy_survived'] += survived
            if gender == 'female':
                family_names[family_name]['female_survived'] += survived
        else:
            family_names[family_name]= {'count':1,
                                        'survived':None,
                                        'num_female':0,
                                        'num_boys':0,
                                        'boy_survived':0,
                                        'female_survived':0,
                                        'members':{ data.Name[idx]: {'id':idx, 'train':True} }
                                       }
            family_names[family_name]['count'] = 1
            family_names[family_name]['survived'] = survived # increment how many family members survived
            family_names[family_name]['num_female'] = 1 if gender == 'female' else 0
            family_names[family_name]['num_boys'] = 1 if boy else 0
            if boy:
                family_names[family_name]['boy_survived'] = survived
            if gender == 'female':
                family_names[family_name]['female_survived'] = survived

                
                
    return family_names

def get_test_family_data(data): # update train data if person in test data 
    test_family_names = {} # test family data 
    
    for idx in range(len(data)):
        full_name = data.Name[idx]
        gender = data.Sex[idx]
        boy = isBoy({'name':full_name, 'gender': gender})
        full_name = full_name.split()

        family_name = full_name[0][:-1]
        
        if family_name in test_family_names: # if family name already in dict, then increment family count
            test_family_names[family_name]['members'].update({data.Name[idx]:{'id':idx,
                                                                   'train':False,
                                                                   'in_train': False,
                                                                   'new_member': False
                                                                   } 
                                                  })
            test_family_names[family_name]['count'] += 1
            test_family_names[family_name]['num_female'] += 1 if gender == 'female' else 0
            test_family_names[family_name]['num_boys'] += 1 if boy else 0
            
            if family_name in family_names_data: # if test family name already in train
                if data.Name[idx] in family_names_data[family_name]['members']: # if the test person already exists in train set!
                    test_family_names[family_name]['members'][data.Name[idx]]['in_train'] = True
                else: # Test person is a new addition to the family
                    test_family_names[family_name]['members'][data.Name[idx]]['new_member']  = True
            else: # Test person is completely new.
                test_family_names[family_name]['members'][data.Name[idx]]['in_train'] = False
                test_family_names[family_name]['members'][data.Name[idx]]['new_member'] = False
         
        else:
            test_family_names[family_name]= {'count':1,
                                        'num_female':0,
                                        'num_boys':0,
                                        'members':{data.Name[idx]:{'id':idx,
                                                                   'train':False,
                                                                   'in_train': False,
                                                                   'new_member': False
                                                                   } 
                                                  },
                                       }
            test_family_names[family_name]['count'] = 1
            test_family_names[family_name]['num_female'] = 1 if gender == 'female' else 0
            test_family_names[family_name]['num_boys'] = 1 if boy else 0
          
    
    return test_family_names

    

# Reading Training Data

In [6]:
# Reading Training Data
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [7]:
family_names_data = get_family_data(train_data)

In [8]:
# Reading Test Data
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head() # 418 test passenggers

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [9]:
test_family_data = get_test_family_data(test_data)

# Processing Training Data

In [10]:
def predict(person):
    '''
    person (dict) contains their full name, and gender
    '''
    full_name = person['full_name']
    family_name = full_name.split()[0][:-1]
    gender = person['gender']
    boy = isBoy({'name':full_name, 'gender':gender})
    num_boys = family_names_data[family_name]['num_boys'] # number of boys in the family
    boys_lived = family_names_data[family_name]['boy_survived']
    
    num_female = family_names_data[family_name]['num_female']
    female_lived = family_names_data[family_name]['female_survived']
    
    survive = False
    
    if gender == 'male':
        if boy:
            if (num_boys == boys_lived) and (num_female==female_lived): # if boy, and if all the boys and females in family live, then the boy might live
                survive = True
            else: # Else if any boys or females in family dies, then the boy is most likely not going to survive
                survive = False

            return survive
        else:
            return survive # default is all men do not survive
    
    if gender == 'female':
        if (female_lived == 0) and (boys_lived==0): # female does not survive iff every female and boys in family does not survive
            survive = False
        else:
            survive = True # by default, chances are, female will live
        
        return survive
    

def test_predict(person): # prediction
    '''
    person (dict) contains full name and gender
    '''
    
    full_name = person['full_name']
    family_name = full_name.split()[0][:-1]
    gender = person['gender']
    boy = isBoy({'name':full_name, 'gender':gender})
    
    survive = False
    
    if (test_family_data[family_name]['members'][person['full_name']]['new_member'] == True):
        # If family name of test in train, then determine surviviablity via train family stats and algorithm        
        num_boys = family_names_data[family_name]['num_boys'] # number of boys in the family
        boys_lived = family_names_data[family_name]['boy_survived']
    
        num_female = family_names_data[family_name]['num_female']
        female_lived = family_names_data[family_name]['female_survived']
    
        if gender == 'male':
            if boy:
                if (num_boys == boys_lived) and (num_female==female_lived): # if boy, and if all the boys and females in family live, then the boy might live
                    survive = True
                else: # Else if any boys or females in family dies, then the boy is most likely not going to survive
                    survive = False

                return survive
            else:
                return survive # default is all men do not survive

        if gender == 'female':
            if (female_lived == 0) and (boys_lived==0): # female does not survive iff every female and boys in family does not survive
                survive = False
            else:
                survive = True # by default, chances are, female will live

            return survive
    
    else: # new family
        num_boys = test_family_data[family_name]['num_boys'] # number of boys in the family
    
        num_female = test_family_data[family_name]['num_female']
    
        if gender == 'male':
            if boy:
                if num_female > 0: # Assume boy is travelling with either a sister, or a mother. Then most likely he will be on boat with them
                    survive = True
                elif num_female == 0: # Assumes the boy has no sisters, or nanny
                    survive = True
                elif num_boys > 1: # Assume boy has another brother
                    survive = False
                    
                return survive
            else:
                survive = False 
                return survive # default is all men do not survive

        if gender == 'female':
            if (num_boys > 0) or (num_female) > 1: # Assumes this female is a mother, or nanny or sister
                survive = True
            else: # Assumes this female does not have any children, then she may not survive.
                
                survive = True # by default, chances are, female will live
            
            return survive
    
    
    

In [11]:
train_pred = {'PassengerId':[], 'Survived':[]} # the dictionary of training data survival predictions

train_correct = 0

for idx in range(len(train_data)):
    fn = train_data.Name[idx] # full name of person
    gender = train_data.Sex[idx] # sex
    person = {'full_name':fn, 'gender':gender}
    pred = predict(person)
    ans = train_data.Survived[idx]
    train_correct += 1 if pred == ans else 0
    train_pred['PassengerId'].append(idx)
    train_pred['Survived'].append(pred)

acc = train_correct/len(train_data)*100
print(f'Train accuracy is {acc:.2f}')

Train accuracy is 89.11


In [12]:
test_pred = {'PassengerId':[], 'Survived':[]} # the dictionary of training data survival predictions


for idx in range(len(test_data)):
    fn = test_data.Name[idx] # full name of person
    gender = test_data.Sex[idx] # sex
    person = {'full_name':fn, 'gender':gender}
    
    pred = test_predict(person)
    pred = 1 if pred == True else 0
    test_pred['PassengerId'].append(idx)
    test_pred['Survived'].append(pred)

#test_pred['Survived']

In [13]:
## Saving model test prediction to csv for competition submission

# Getting todays date for naming csv output file
today = date.today()

# Creating output as pandas dataframe
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': test_pred['Survived']})

# Converting dataframes to csv for competition submission
output.to_csv(f'submission-{today}.csv', index=False)

# CSV saving confirmation
print(f"Model prediction file: submission-{today}.csv successfully saved!")

Model prediction file: submission-2022-06-04.csv successfully saved!
